# Using Nested CDTs with CTX

In [2]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;

IJava.getKernelInstance().getMagics().registerMagics(Shell.class);

In [3]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client</artifactId>
    <version>5.0.0</version>
  </dependency>
</dependencies>

In [4]:
import com.aerospike.client.AerospikeClient;

AerospikeClient client = new AerospikeClient("localhost", 3000);
System.out.println("Initialized the client and connected to the cluster.");

Initialized the client and connected to the cluster.


In [91]:
import com.aerospike.client.Key;
import com.aerospike.client.Bin;
import com.aerospike.client.policy.ClientPolicy;
import com.aerospike.client.Record;
import com.aerospike.client.Operation;
import com.aerospike.client.Value;
import com.aerospike.client.cdt.ListOperation;
import com.aerospike.client.cdt.ListPolicy;
import com.aerospike.client.cdt.ListOrder;
import com.aerospike.client.cdt.ListWriteFlags;
import com.aerospike.client.cdt.MapOperation;
import com.aerospike.client.cdt.MapPolicy;
import com.aerospike.client.cdt.MapOrder;
import com.aerospike.client.cdt.MapWriteFlags;
import com.aerospike.client.Value.IntegerValue;

import java.util.ArrayList;
import java.util.Arrays;
import java.util.HashMap;
import java.util.List;
import java.util.Map;


ArrayList<Value> list1 = new ArrayList<Value>(Arrays.asList(Value.get(1)));

ArrayList<Value> list2 = new ArrayList<Value>(Arrays.asList(Value.get(1), Value.get(2)));

ArrayList<Value> list3 = new ArrayList<Value>(Arrays.asList(Value.get(1), Value.get(2), Value.get(3)));

ArrayList<Value> list4 = new ArrayList<Value>(Arrays.asList(Value.get(1), Value.get(2), Value.get(3), Value.get(4)));

// Create Map of String->List

HashMap <Value, Value> mapBinVal = new HashMap <Value, Value>();
mapBinVal.put(Value.get("1"), Value.get(list1));
mapBinVal.put(Value.get("2"), Value.get(list2));
mapBinVal.put(Value.get("3"), Value.get(list3));
mapBinVal.put(Value.get("4"), Value.get(list4));


// Put data in Aerospike, get the data, and print it

//MapPolicy mapPolicy = new MapPolicy(MapOrder.KEY_ORDERED, MapWriteFlags.DEFAULT);

Bin bin1 = new Bin("mapBin", mapBinVal);

Key key = new Key("test", "test", "test");

Record record = client.operate(null, key,
    Operation.put(bin1),
    Operation.get()
    //MapOperation.putItems(null, "mapBin", mapBinVal)
    );

System.out.println(record );

(gen:20),(exp:388627832),(bins:(mapBin:{1=[1], 2=[1, 2], 3=[1, 2, 3], 4=[1, 2, 3, 4]}))


In [92]:
import com.aerospike.client.cdt.CTX;
import com.aerospike.client.cdt.MapReturnType;

record = client.operate(null, key, 
    ListOperation.getByIndex("mapBin", 0, MapReturnType.VALUE, CTX.mapKey(Value.get("4")))
    );

System.out.println(record);


(gen:20),(exp:388627832),(bins:(mapBin:1))


In [95]:
record = client.operate(null, key, 
    ListOperation.appendItems("mapBin", list3, CTX.mapKeyCreate(Value.get("4"), MapOrder.UNORDERED)),
    ListOperation.appendItems("mapBin", list3, CTX.mapKeyCreate(Value.get("0"), MapOrder.UNORDERED)),
    ListOperation.append("mapBin", Value.get(0), CTX.mapKeyCreate(Value.get("1"), MapOrder.UNORDERED)),
    ListOperation.append("mapBin", Value.get(0), CTX.mapKeyCreate(Value.get("6"), MapOrder.UNORDERED))

    );

Record record = client.operate(null, key,
    Operation.get()
    );

System.out.println(record );


(gen:22),(exp:388627875),(bins:(mapBin:{0=[1, 2, 3, 1, 2, 3], 1=[1, 0, 0], 2=[1, 2], 3=[1, 2, 3], 4=[1, 2, 3, 4, 1, 2, 3, 1, 2, 3], 6=[0]}))
